<span style="font-size: 30px;">EJERCICIO DE CLASIFICACIÓN BAYESIANA </span>

A partir del dataset de pesos y alturas, construir manualmente (sin usar sklearn) un Clasificador Bayesiano LDA. Para calcular los parámetros, utilizar las fórmulas compartidas en la notebook correspondiente.

<span style="font-size: 24px;">Preparación de los datos</span>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_train=pd.read_csv('alturas-pesos-mils-train.csv')
data_test=pd.read_csv('alturas-pesos-mils-test.csv')

In [ ]:
#Paso la infromación del .csv a vectores para que sea más fácil manipular la información
genero_train= data_train['Genero']
wh_train=data_train.iloc[:, 2:4].values

genero_test= data_test['Genero']
wh_test=data_test.iloc[:, 2:4].values

In [ ]:
wh_train

In [ ]:
wh_test

<span style="font-size: 24px;">A. Clasificador manual</span>

In [ ]:
#Calculo las priors: la probabilidad de que una persona sea hombre o mujer
total=len(genero_train)
cant_hombres=len(genero_train[genero_train=='Hombre'])
cant_mujeres=len(genero_train[genero_train=='Mujer'])
suma=cant_mujeres + cant_hombres #esto es útil para asegurarse de que no hay otro tipo de dato
print("TOTAL:", total, "\nMUJERES",  cant_mujeres, "\nHOMBRES:", cant_hombres, "\nSUMA:", suma)
prior_hombre=cant_hombres/total
prior_mujer=cant_mujeres/total
print("PRIOR HOMBRES:", prior_hombre, "\nPRIOR MUJERES:", prior_mujer)

\begin{equation}
  P(C_{i} | w, h) = \frac{P( w, h | C_{i})\times P(C_{i})}{P(w,h)}
\end{equation}

Veo que la altura va de 140 a 200cm, y los pesos de 20 a 140kg.

In [ ]:
#Propiedades para los bines
minw=20
maxw=130
minh=135
maxh=225
cantidad_bines=100
ancho_bines_peso=(maxw-minw)/cantidad_bines
ancho_bines_altura=(maxh-minh)/cantidad_bines

OPCIÓN 1: diseñamos un primer clasificador que si llega a un dato que no se puede clasificar, no lo toma en consideración

In [ ]:
#OPCIÓN 1 - Si llega a un dato que no se puede clasificar, no lo toma en consideración
def probabilidad_wh_A(wh_vector, genero):
  cont_wh=0
  cont_dado_genero=0
  if(genero=='Hombre'):
    cant_genero=cant_hombres
  else:
    cant_genero=cant_mujeres

  peso=wh_vector[0]
  altura=wh_vector[1]

  #identifico en qué bin está
  binW=int((peso-minw)/ancho_bines_peso)
  binH=int((altura-minh)/ancho_bines_altura)

  infW=minw+binW*ancho_bines_peso #límite inferior de peso del bin
  supW=minw+(binW+1)*ancho_bines_peso #límite superior de peso del bin
  infH=minh+binH*ancho_bines_altura #límite inferior de altura del bin
  supH=minh+(binH+1)*ancho_bines_altura #límite superior de altura del bin

  #cuento cuántas muestras hay en ese bin
  for i in range(total):
    if (infW<= wh_train[i, 0] <supW) and (infH<= wh_train[i,1] <supH):
      cont_wh+=1
      if (genero_train[i]==genero):
        cont_dado_genero+=1

  if cont_wh==0:
    clasificado=False
    proba=0
    likelihood=0
  else:
      clasificado=True
      proba=cont_wh/total
      likelihood=cont_dado_genero/cant_genero
  
  return proba, likelihood, clasificado

In [ ]:
def clasificador_manual_A(wh_vector, genero_vector):
    cant_noClasificados=0
    prediccion=[] #en este vector guardo los resultados de la predicción

    #Recorre todos los elementos del array y los clasifica 
    for i in range(len(genero_vector)):
        #print("Evalúo el elemento ",i, "---PESO=", peso_train[i], "---ALTURA=", altura_train[i])
        proba_wh, likelihood_hombre,clasificado= probabilidad_wh_A(wh_vector[i], 'Hombre')

        if clasificado==True:
            #Calculo, por medio del teorema de Bayes, la probabilidad de que sea hombre
            hombre_dado_wh= (likelihood_hombre * prior_hombre)/proba_wh
            if(hombre_dado_wh>0.5):
                prediccion.append(1) #de esta forma, HOMBRE=1 MUJER=0
            else:
                prediccion.append(0)
        if clasificado==False:
            cant_noClasificados+=1
            print("La muestra ", i, "no se pudo clasificar")
    
    print("\n\nHUBO ", cant_noClasificados, " ELEMENTOS QUE NO PUDIERON SER CLASIFICADOS")
    #una vez que termina, convierto la lista con las predicciones en array
    prediccion=np.array(prediccion)

    return prediccion

OPCIÓN 2: En este segundo clasificador, si hay un dato que no puede ser clasificado, se compara esa muestra con los bins de alrededor del bin que le corresponde. Se ampliará el área de comparación hasta que la muestra pueda ser clasificada.

In [ ]:
def probabilidad_wh_B(wh_vector, genero, indice):
  cont_wh=0
  k=-1
  cont_dado_genero=0
  if(genero=='Hombre'):
    cant_genero=cant_hombres
  else:
    cant_genero=cant_mujeres

  peso=wh_vector[0]
  altura=wh_vector[1]
  #identifico en qué bin está
  binW=int((peso-minw)/ancho_bines_peso)
  binH=int((altura-minh)/ancho_bines_altura)

  while(cont_wh==0): #Si encontró al menos un elemento en el mismo bin, sigue. Sino, se aplía el bin y se vuelve a hacer la búsqueda.
    k+=1
    infW=minw+(binW-k)*ancho_bines_peso #límite inferior de peso del bin
    supW=minw+(binW+1+k)*ancho_bines_peso #límite superior de peso del bin
    infH=minh+(binH-k)*ancho_bines_altura #límite inferior de altura del bin
    supH=minh+(binH+1+k)*ancho_bines_altura #límite superior de altura del bin

    #cuento cuántas muestras hay en ese bin
    for i in range(total):
      if (infW<= wh_train[i, 0] <supW) and (infH<= wh_train[i,1] <supH):
        cont_wh+=1
        if (genero_train[i]==genero):
          cont_dado_genero+=1
  
  if k!=0:
    print("Para la muestra ", indice, "Se amplió ", k, "veces el espacio de comparación")

  proba=cont_wh/total
  likelihood=cont_dado_genero/cant_genero

  return proba, likelihood

In [ ]:
def clasificador_manual_B(wh_vector, genero_vector):
    prediccion=np.zeros(len(genero_vector)) #en este vector guardo los resultados de la predicción

    #Recorre todos los elementos del array y los clasifica 
    for i in range(len(genero_vector)):
        #print("Evalúo el elemento ",i, "---PESO=", peso_train[i], "---ALTURA=", altura_train[i])
        proba_wh, likelihood_hombre= probabilidad_wh_B(wh_vector[i], 'Hombre', i)
        hombre_dado_wh= (likelihood_hombre * prior_hombre)/proba_wh
        if(hombre_dado_wh>0.5):
            prediccion[i]=1 

    return prediccion

<span style="font-size: 20px;"> A.1. Imágenes de entrenamiento </span>

In [ ]:
#con la primera opción
prediccion_manual_train_A=clasificador_manual_A(wh_train, genero_train)

In [ ]:
prediccion_manual_train_B=clasificador_manual_B(wh_train, genero_train)

<span style="font-size: 20px;"> A. 2. Imágenes de testeo </span>

In [ ]:
prediccion_manual_test_A=clasificador_manual_A(wh_test, genero_test)

In [ ]:
prediccion_manual_test_B=clasificador_manual_B(wh_test, genero_test)

<span style="font-size: 20px;"> A. 3. Métricas </span>

In [ ]:
def binarizar_genero(vector):
    binarizado=np.zeros(len(vector))
    for i in range(len(vector)):
        if (vector[i]=='Hombre'):
            binarizado[i]=1
    return binarizado

In [ ]:
#Paso los valores reales a números
genero_train_binario=binarizar_genero(genero_train)
genero_test_binario=binarizar_genero(genero_test)

In [ ]:
etiquetas=['Hombre', 'Mujer']

Diseñamos nuestra propiafunción para calcular el accuracy, según la ecuación: 

\begin{equation}
 \frac{TP +TN}{TP + TN + FP + FN}
\end{equation}



In [ ]:
def accuracy_manual(prediccion, vector_real):
    TP=0 #que prediga que es hombre (1) y realmente lo sea
    TN=0 #que prediga que es mujer (0) y realmente lo sea 

    FN=0 #que prediga que es mujer (0) y en realidad es hombre (1)
    FP=0 #que prediga que es hombre (1) y en realidad es muje (0)
    for i in range(len(vector_real)):
        if (prediccion[i]==1 and vector_real[i]==1):
            TP+=1
        if (prediccion[i]==0 and vector_real[i]==0):
            TN+=1
    accuracy=(TP + TN)/(len(vector_real))
    return accuracy 

In [ ]:
accuracy_manual_train=accuracy_manual(prediccion_manual_train_B, genero_train_binario)
accuracy_manual_test=accuracy_manual(prediccion_manual_test_B, genero_test_binario)
print("ACCURACY DEL MODELO HECHO MANUALMENTE")
print("\tConjunto de entrenamiento: ", accuracy_manual_train, "\n\tConjunto de testeo: ", accuracy_manual_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

In [ ]:
cm_manual_train=confusion_matrix(genero_train_binario, prediccion_manual_train_B)
cm_manual_test=confusion_matrix(genero_test_binario, prediccion_manual_test_B)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Resultados con clasificador hecho "a mano"')

sns.heatmap(cm_manual_train, annot=True, fmt="d", cmap="Blues", ax=ax[0],  xticklabels= etiquetas, yticklabels= etiquetas)
ax[0].set_title("Entrenamiento")
ax[0].set_xlabel("Predicción")
ax[0].set_ylabel("Real")

sns.heatmap(cm_manual_test, annot=True, fmt="d", cmap="Oranges", ax=ax[1], xticklabels= etiquetas, yticklabels= etiquetas)
ax[1].set_title("Testeo")
ax[1].set_xlabel("Predicción")
ax[1].set_ylabel("Real")
plt.show()

<span style="font-size: 24px;"> B. Clasificador de SciKit-Learn </span>

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf_sklearn = GaussianNB()
clf_sklearn.fit(wh_train, genero_train_binario)  # Entrenar el modelo


<span style="font-size: 20px;"> B. 1. Imágenes de entrenamiento </span>

In [ ]:
prediccion_sklearn_train = clf_sklearn.predict(wh_train)

<span style="font-size: 20px;"> B. 2. Imágenes de testeo </span>

In [ ]:
prediccion_sklearn_test = clf_sklearn.predict(wh_test)

<span style="font-size: 20px;"> B. 3. Métricas </span>

Calculamos el accuracy con la función de la propia librería Scikit-Learn

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_sklearn_train = accuracy_score(genero_train_binario, prediccion_sklearn_train )
accuracy_sklearn_test = accuracy_score(genero_test_binario, prediccion_sklearn_test)


In [ ]:
print("ACCURACY DEL MODELO ENTRENADO CON SCIKIT-LEARN")
print("\tConjunto de entrenamiento: ", accuracy_sklearn_train, "\n\tConjunto de testeo: ", accuracy_sklearn_test)

Para dar una mayor idea sobre el desempeño del modelo, mostramos las matrices de confusión.

In [ ]:
cm_sklearn_train=confusion_matrix(genero_train_binario, prediccion_sklearn_train)
cm_sklearn_test=confusion_matrix(genero_test_binario, prediccion_sklearn_test)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Resultados con clasificador hecho de SciKit-Learn')

sns.heatmap(cm_sklearn_train, annot=True, fmt="d", cmap="Blues", ax=ax[0],  xticklabels= etiquetas, yticklabels= etiquetas)
ax[0].set_title("Entrenamiento")
ax[0].set_xlabel("Predicción")
ax[0].set_ylabel("Real")

sns.heatmap(cm_sklearn_test, annot=True, fmt="d", cmap="Oranges", ax=ax[1], xticklabels= etiquetas, yticklabels= etiquetas)
ax[1].set_title("Testeo")
ax[1].set_xlabel("Predicción")
ax[1].set_ylabel("Real")
plt.show()

<span style="font-size: 24px;"> C. Clasificación con método de Expansión Cuadrática </span>

In [ ]:
#PREPARO EL VECTOR CON LOS DATOS
def expandir_datos(wh_vector):
    datos_extension=np.zeros([len(wh_vector), 5])
    for i in range(len(wh_vector)):
        peso=wh_train[i, 0]
        altura=wh_train[i, 1]
        datos_extension[i, 0]=peso
        datos_extension[i, 1]=peso**2
        datos_extension[i, 2]=altura
        datos_extension[i, 3]=altura**2
        datos_extension[i, 4]=peso*altura
    return datos_extension

In [ ]:
datos_expansion_train= expandir_datos(wh_train)
datos_expansion_test=expandir_datos(wh_test)

In [ ]:
#Entrenamos el modelo
clf_expansion = GaussianNB()
clf_expansion.fit(datos_expansion_train, genero_train_binario)  

<span style="font-size: 20px;"> C. 1. Imágenes de entrenamiento </span>

In [ ]:
prediccion_expansion_train = clf_expansion.predict(datos_expansion_train)

<span style="font-size: 20px;"> C. 2. Imágenes de testeo </span>

In [ ]:
prediccion_expansion_test = clf_expansion.predict(datos_expansion_test)

<span style="font-size: 20px;"> C. 3. Métricas </span>

In [ ]:
accuracy_expansion_train = accuracy_score(genero_train_binario, prediccion_expansion_train )
accuracy_expansion_test = accuracy_score(genero_test_binario, prediccion_expansion_test)
print("ACCURACY DEL MODELO ENTRENADO CON DATOS EXPANDIDOS")
print("\tConjunto de entrenamiento: ", accuracy_expansion_train, "\n\tConjunto de testeo: ", accuracy_expansion_test)


In [ ]:
cm_expansion_train=confusion_matrix(genero_train_binario, prediccion_expansion_train)
cm_expansion_test=confusion_matrix(genero_test_binario, prediccion_expansion_test)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Resultados con clasificador entrenado con Expansión Cuadrática')
sns.heatmap(cm_expansion_train, annot=True, fmt="d", cmap="Blues", ax=ax[0],  xticklabels= etiquetas, yticklabels= etiquetas)
ax[0].set_title("Entrenamiento")
ax[0].set_xlabel("Predicción")
ax[0].set_ylabel("Real")

sns.heatmap(cm_expansion_test, annot=True, fmt="d", cmap="Oranges", ax=ax[1], xticklabels= etiquetas, yticklabels= etiquetas)
ax[1].set_title("Testeo")
ax[1].set_xlabel("Predicción")
ax[1].set_ylabel("Real")
plt.show()